## 1. Setup and Imports

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
import joblib

# Scikit-learn imports
from sklearn.model_selection import train_test_split, cross_val_score, GroupKFold
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.decomposition import PCA
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Models
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import Ridge, ElasticNet
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, HistGradientBoostingRegressor

# MLflow
import mlflow
import mlflow.sklearn

warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8-whitegrid')

# Project paths
PROJECT_ROOT = Path('/workspaces/Recommendation-system')
DATA_PROCESSED = PROJECT_ROOT / 'data' / 'processed'
MODELS_DIR = PROJECT_ROOT / 'models'
REPORTS_FIGURES = PROJECT_ROOT / 'reports' / 'figures'
MLRUNS_DIR = PROJECT_ROOT / 'mlruns'

# Create directories
MODELS_DIR.mkdir(parents=True, exist_ok=True)

# Constants
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

print("Setup complete!")

## 2. Load Data

In [ ]:
# Load consolidated dataset
df = pd.read_csv(DATA_PROCESSED / 'consolidated.csv')
print(f"Dataset shape: {df.shape}")
print(f"Columns: {df.columns.tolist()}")
df.head()

In [ ]:
# Define features and target
NUMERIC_FEATURES = ['rainfall_mm', 'pesticides_tonnes', 'avg_temp']
CATEGORICAL_FEATURES = ['crop', 'country']
TARGET = 'yield'

print(f"Numeric features: {NUMERIC_FEATURES}")
print(f"Categorical features: {CATEGORICAL_FEATURES}")
print(f"Target: {TARGET}")

## 3. Principal Component Analysis (PCA)

PCA is used to identify key variables driving variance in the context space.

In [ ]:
# Prepare data for PCA (numeric features only, standardized)
X_numeric = df[NUMERIC_FEATURES].copy()

# Standardize
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_numeric)

# Fit PCA
pca = PCA()
pca.fit(X_scaled)

print("PCA Results:")
print(f"Number of components: {pca.n_components_}")
print(f"\nExplained variance ratio:")
for i, var in enumerate(pca.explained_variance_ratio_):
    print(f"  PC{i+1}: {var:.4f} ({var*100:.2f}%)")
print(f"\nCumulative explained variance: {pca.explained_variance_ratio_.cumsum()}")

In [ ]:
# Plot explained variance
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Scree plot
components = range(1, len(pca.explained_variance_ratio_) + 1)
axes[0].bar(components, pca.explained_variance_ratio_, alpha=0.7, label='Individual')
axes[0].plot(components, pca.explained_variance_ratio_.cumsum(), 'ro-', label='Cumulative')
axes[0].axhline(y=0.95, color='g', linestyle='--', label='95% threshold')
axes[0].set_xlabel('Principal Component')
axes[0].set_ylabel('Explained Variance Ratio')
axes[0].set_title('PCA Explained Variance')
axes[0].legend()
axes[0].set_xticks(components)

# Loadings heatmap
loadings = pd.DataFrame(
    pca.components_.T,
    columns=[f'PC{i+1}' for i in range(pca.n_components_)],
    index=NUMERIC_FEATURES
)
sns.heatmap(loadings, annot=True, cmap='RdBu_r', center=0, ax=axes[1], fmt='.3f')
axes[1].set_title('PCA Loadings Matrix')

plt.tight_layout()
plt.savefig(REPORTS_FIGURES / 'pca_analysis.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# PCA Interpretation
print("=" * 60)
print("PCA INTERPRETATION")
print("=" * 60)

print("\nLoadings Matrix:")
print(loadings.round(3))

print("\n" + "=" * 60)
print("KEY FINDINGS:")
print("=" * 60)

# Find dominant features for each component
for i in range(min(3, pca.n_components_)):
    pc_loadings = loadings.iloc[:, i].abs().sort_values(ascending=False)
    dominant = pc_loadings.index[0]
    print(f"\nPC{i+1} ({pca.explained_variance_ratio_[i]*100:.1f}% variance):")
    print(f"  Dominated by: {dominant}")
    for feat in NUMERIC_FEATURES:
        loading = loadings.loc[feat, f'PC{i+1}']
        direction = 'positive' if loading > 0 else 'negative'
        print(f"  - {feat}: {loading:.3f} ({direction})")

## 4. Train/Test Split

In [ ]:
# Prepare features and target
X = df[NUMERIC_FEATURES + CATEGORICAL_FEATURES]
y = df[TARGET]

print(f"Feature matrix shape: {X.shape}")
print(f"Target shape: {y.shape}")

In [ ]:
# Time-based split: use earlier years for training, later for testing
# First, let's check year distribution
print(f"Year range: {df['year'].min()} - {df['year'].max()}")

# Split at ~80% of data (by year)
year_cutoff = df['year'].quantile(0.8)
print(f"Year cutoff for train/test split: {year_cutoff}")

# Create train/test masks
train_mask = df['year'] <= year_cutoff
test_mask = df['year'] > year_cutoff

X_train = X[train_mask]
X_test = X[test_mask]
y_train = y[train_mask]
y_test = y[test_mask]

print(f"\nTraining set: {len(X_train)} samples ({100*len(X_train)/len(X):.1f}%)")
print(f"Test set: {len(X_test)} samples ({100*len(X_test)/len(X):.1f}%)")
print(f"Training years: {df[train_mask]['year'].min()} - {df[train_mask]['year'].max()}")
print(f"Test years: {df[test_mask]['year'].min()} - {df[test_mask]['year'].max()}")

## 5. Create Preprocessing Pipeline

In [ ]:
# Create preprocessor
numeric_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='constant', fill_value='unknown')),
    ('encoder', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, NUMERIC_FEATURES),
        ('cat', categorical_transformer, CATEGORICAL_FEATURES)
    ]
)

print("Preprocessor created!")

## 6. MLflow Setup and Model Training

In [ ]:
# Setup MLflow
mlflow.set_tracking_uri(str(MLRUNS_DIR))
mlflow.set_experiment("crop_yield_prediction")

print(f"MLflow tracking URI: {mlflow.get_tracking_uri()}")
print(f"Experiment: crop_yield_prediction")

In [ ]:
def evaluate_model(model, X_train, X_test, y_train, y_test):
    """Train and evaluate a model, returning metrics."""
    model.fit(X_train, y_train)
    
    # Training metrics
    y_train_pred = model.predict(X_train)
    train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))
    train_mae = mean_absolute_error(y_train, y_train_pred)
    train_r2 = r2_score(y_train, y_train_pred)
    
    # Test metrics
    y_test_pred = model.predict(X_test)
    test_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))
    test_mae = mean_absolute_error(y_test, y_test_pred)
    test_r2 = r2_score(y_test, y_test_pred)
    
    return {
        'train_rmse': train_rmse,
        'train_mae': train_mae,
        'train_r2': train_r2,
        'test_rmse': test_rmse,
        'test_mae': test_mae,
        'test_r2': test_r2,
        'y_test_pred': y_test_pred
    }

In [ ]:
# Define models to train
models = {
    'Baseline (Mean)': DummyRegressor(strategy='mean'),
    'Ridge Regression': Ridge(alpha=1.0, random_state=RANDOM_STATE),
    'ElasticNet': ElasticNet(alpha=1.0, l1_ratio=0.5, random_state=RANDOM_STATE),
    'Random Forest': RandomForestRegressor(n_estimators=100, max_depth=15, random_state=RANDOM_STATE, n_jobs=-1),
    'Gradient Boosting': GradientBoostingRegressor(n_estimators=100, max_depth=5, learning_rate=0.1, random_state=RANDOM_STATE),
    'HistGradientBoosting': HistGradientBoostingRegressor(max_iter=100, max_depth=10, learning_rate=0.1, random_state=RANDOM_STATE)
}

print(f"Training {len(models)} models...")

In [ ]:
# Train all models with MLflow tracking
results = {}

for name, model in models.items():
    print(f"\nTraining: {name}")
    
    # Create pipeline
    pipeline = Pipeline([
        ('preprocessor', preprocessor),
        ('model', model)
    ])
    
    with mlflow.start_run(run_name=name):
        # Log parameters
        mlflow.log_param('model_type', name)
        mlflow.log_param('numeric_features', NUMERIC_FEATURES)
        mlflow.log_param('categorical_features', CATEGORICAL_FEATURES)
        
        # Log model-specific parameters
        if hasattr(model, 'get_params'):
            params = model.get_params()
            for key, value in params.items():
                if value is not None and not callable(value):
                    mlflow.log_param(f'model_{key}', value)
        
        # Train and evaluate
        metrics = evaluate_model(pipeline, X_train, X_test, y_train, y_test)
        
        # Log metrics
        mlflow.log_metric('train_rmse', metrics['train_rmse'])
        mlflow.log_metric('train_mae', metrics['train_mae'])
        mlflow.log_metric('train_r2', metrics['train_r2'])
        mlflow.log_metric('test_rmse', metrics['test_rmse'])
        mlflow.log_metric('test_mae', metrics['test_mae'])
        mlflow.log_metric('test_r2', metrics['test_r2'])
        
        # Log model
        mlflow.sklearn.log_model(pipeline, 'model')
        
        # Store results
        results[name] = {
            'pipeline': pipeline,
            'metrics': metrics
        }
        
        print(f"  Test RMSE: {metrics['test_rmse']:.2f}")
        print(f"  Test MAE: {metrics['test_mae']:.2f}")
        print(f"  Test R²: {metrics['test_r2']:.4f}")

print("\n✅ All models trained and logged to MLflow!")

## 7. Model Comparison

In [ ]:
# Create comparison DataFrame
comparison_data = []
for name, result in results.items():
    metrics = result['metrics']
    comparison_data.append({
        'Model': name,
        'Train RMSE': metrics['train_rmse'],
        'Test RMSE': metrics['test_rmse'],
        'Train MAE': metrics['train_mae'],
        'Test MAE': metrics['test_mae'],
        'Train R²': metrics['train_r2'],
        'Test R²': metrics['test_r2']
    })

comparison_df = pd.DataFrame(comparison_data)
comparison_df = comparison_df.sort_values('Test R²', ascending=False)

print("Model Comparison (sorted by Test R²):")
comparison_df

In [ ]:
# Visualize model comparison
fig, axes = plt.subplots(1, 3, figsize=(16, 5))

# RMSE comparison
x = range(len(comparison_df))
width = 0.35
axes[0].bar([i - width/2 for i in x], comparison_df['Train RMSE'], width, label='Train', alpha=0.8)
axes[0].bar([i + width/2 for i in x], comparison_df['Test RMSE'], width, label='Test', alpha=0.8)
axes[0].set_xticks(x)
axes[0].set_xticklabels(comparison_df['Model'], rotation=45, ha='right')
axes[0].set_ylabel('RMSE')
axes[0].set_title('RMSE Comparison')
axes[0].legend()

# MAE comparison
axes[1].bar([i - width/2 for i in x], comparison_df['Train MAE'], width, label='Train', alpha=0.8)
axes[1].bar([i + width/2 for i in x], comparison_df['Test MAE'], width, label='Test', alpha=0.8)
axes[1].set_xticks(x)
axes[1].set_xticklabels(comparison_df['Model'], rotation=45, ha='right')
axes[1].set_ylabel('MAE')
axes[1].set_title('MAE Comparison')
axes[1].legend()

# R² comparison
axes[2].bar([i - width/2 for i in x], comparison_df['Train R²'], width, label='Train', alpha=0.8)
axes[2].bar([i + width/2 for i in x], comparison_df['Test R²'], width, label='Test', alpha=0.8)
axes[2].set_xticks(x)
axes[2].set_xticklabels(comparison_df['Model'], rotation=45, ha='right')
axes[2].set_ylabel('R²')
axes[2].set_title('R² Comparison')
axes[2].legend()

plt.tight_layout()
plt.savefig(REPORTS_FIGURES / 'model_comparison.png', dpi=150, bbox_inches='tight')
plt.show()

## 8. Select Best Model

In [ ]:
# Select best model based on test R²
best_model_name = comparison_df.iloc[0]['Model']
best_model = results[best_model_name]['pipeline']
best_metrics = results[best_model_name]['metrics']

print("=" * 60)
print("BEST MODEL SELECTED")
print("=" * 60)
print(f"\nModel: {best_model_name}")
print(f"\nTest Metrics:")
print(f"  RMSE: {best_metrics['test_rmse']:.2f}")
print(f"  MAE: {best_metrics['test_mae']:.2f}")
print(f"  R²: {best_metrics['test_r2']:.4f}")

## 9. Feature Importance Analysis

In [ ]:
# Get feature names after preprocessing
def get_feature_names(preprocessor, numeric_features, categorical_features):
    """Extract feature names from fitted preprocessor."""
    feature_names = list(numeric_features)  # Numeric features first
    
    # Get one-hot encoded feature names
    cat_encoder = preprocessor.named_transformers_['cat'].named_steps['encoder']
    cat_feature_names = cat_encoder.get_feature_names_out(categorical_features)
    feature_names.extend(cat_feature_names)
    
    return feature_names

# Get feature names
fitted_preprocessor = best_model.named_steps['preprocessor']
feature_names = get_feature_names(fitted_preprocessor, NUMERIC_FEATURES, CATEGORICAL_FEATURES)
print(f"Total features after encoding: {len(feature_names)}")

In [ ]:
# Extract feature importance (for tree-based models)
model_estimator = best_model.named_steps['model']

if hasattr(model_estimator, 'feature_importances_'):
    importances = model_estimator.feature_importances_
    
    # Create DataFrame
    importance_df = pd.DataFrame({
        'feature': feature_names[:len(importances)],
        'importance': importances
    }).sort_values('importance', ascending=False)
    
    # Plot top 20 features
    plt.figure(figsize=(12, 8))
    top_n = min(20, len(importance_df))
    plt.barh(importance_df['feature'][:top_n][::-1], importance_df['importance'][:top_n][::-1])
    plt.xlabel('Feature Importance')
    plt.title(f'Top {top_n} Feature Importances ({best_model_name})')
    plt.tight_layout()
    plt.savefig(REPORTS_FIGURES / 'feature_importance.png', dpi=150, bbox_inches='tight')
    plt.show()
    
    print("\nTop 10 Features:")
    print(importance_df.head(10))
else:
    print(f"{best_model_name} does not have feature_importances_ attribute")

## 10. Prediction Analysis

In [ ]:
# Actual vs Predicted plot
y_test_pred = best_metrics['y_test_pred']

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Scatter plot
axes[0].scatter(y_test, y_test_pred, alpha=0.3, s=10)
axes[0].plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2, label='Perfect prediction')
axes[0].set_xlabel('Actual Yield (hg/ha)')
axes[0].set_ylabel('Predicted Yield (hg/ha)')
axes[0].set_title(f'Actual vs Predicted ({best_model_name})')
axes[0].legend()

# Residuals distribution
residuals = y_test - y_test_pred
axes[1].hist(residuals, bins=50, edgecolor='black', alpha=0.7)
axes[1].axvline(0, color='red', linestyle='--', linewidth=2)
axes[1].set_xlabel('Residual (Actual - Predicted)')
axes[1].set_ylabel('Frequency')
axes[1].set_title('Residuals Distribution')

plt.tight_layout()
plt.savefig(REPORTS_FIGURES / 'prediction_analysis.png', dpi=150, bbox_inches='tight')
plt.show()

print(f"\nResiduals Statistics:")
print(f"  Mean: {residuals.mean():.2f}")
print(f"  Std: {residuals.std():.2f}")
print(f"  Min: {residuals.min():.2f}")
print(f"  Max: {residuals.max():.2f}")

## 11. Save Final Model

In [ ]:
# Save the best model pipeline
model_path = MODELS_DIR / 'model_pipeline.joblib'
joblib.dump(best_model, model_path)

print(f"Model saved to: {model_path}")
print(f"File size: {model_path.stat().st_size / 1024 / 1024:.2f} MB")

In [ ]:
# Save model metadata
metadata = {
    'model_name': best_model_name,
    'numeric_features': NUMERIC_FEATURES,
    'categorical_features': CATEGORICAL_FEATURES,
    'target': TARGET,
    'test_rmse': best_metrics['test_rmse'],
    'test_mae': best_metrics['test_mae'],
    'test_r2': best_metrics['test_r2'],
    'training_samples': len(X_train),
    'test_samples': len(X_test),
    'supported_crops': df['crop'].unique().tolist(),
    'supported_countries': df['country'].unique().tolist()
}

# Save as JSON
import json
metadata_path = MODELS_DIR / 'model_metadata.json'
with open(metadata_path, 'w') as f:
    json.dump(metadata, f, indent=2)

print(f"Metadata saved to: {metadata_path}")

In [ ]:
# Verify model can be loaded and used
loaded_model = joblib.load(model_path)

# Test prediction
test_input = pd.DataFrame([{
    'rainfall_mm': 1000,
    'pesticides_tonnes': 5000,
    'avg_temp': 20,
    'crop': 'Wheat',
    'country': 'India'
}])

prediction = loaded_model.predict(test_input)
print(f"\nTest prediction for Wheat in India:")
print(f"  Input: rainfall=1000mm, pesticides=5000t, temp=20°C")
print(f"  Predicted yield: {prediction[0]:.0f} hg/ha")

## 12. MLflow Summary

To view MLflow UI, run in terminal:
```bash
mlflow ui --backend-store-uri /workspaces/Recommendation-system/mlruns
```

In [ ]:
print("=" * 60)
print("TRAINING SUMMARY")
print("=" * 60)
print(f"\nExperiment: crop_yield_prediction")
print(f"MLflow tracking directory: {MLRUNS_DIR}")
print(f"\nModels trained: {len(models)}")
for name in models.keys():
    print(f"  - {name}")
print(f"\nBest model: {best_model_name}")
print(f"Best test R²: {best_metrics['test_r2']:.4f}")
print(f"\nModel artifact: {model_path}")
print(f"\n✅ Modeling complete! Ready for API integration.")